In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
filename="../dataset/cat.ppm"

img = cv2.imread(filename)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)

**Image normalization for CNN**
- Normalize an RGB image for CNN detection while preserving object colors.
- Normalization must be applied simultaneously to all 3 RGB channels of the image, because per-channel normalization would change object colors.

Normalization formula for each pixel:
$$
\text{normalized\_pixel} = \frac{\text{pixel} - \mu}{\max(\sigma, \frac{1}{\sqrt{N}})}
$$

where:
- $\mu$ is the mean computed over all pixels and all channels
- $\sigma$ is the standard deviation computed over all pixels and all channels
- $N = \text{height} \times \text{width} \times 3$ (total number of values)

In [ ]:
mean = 0.0  
N = 24 * 24 * 3 
for i in range(24):
    for j in range(24):
        for c in range(3):
            mean += float(img[i][j][c])
mean = mean / N

std_dev = 0.0
for i in range(24):
    for j in range(24):
        for c in range(3):
            std_dev += (float(img[i][j][c]) - mean) ** 2
std_dev = np.sqrt(std_dev / N)

print(f"Moyenne globale: {mean:.2f}")
print(f"Écart-type global: {std_dev:.2f}")

img_norm = (img - mean) / max(std_dev, 1/np.sqrt(N))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(img)
axes[0].set_title("Image originale")
axes[0].axis('on')

display_img = img_norm.copy()
display_img = ((display_img - display_img.min()) / (display_img.max() - display_img.min()) * 255).astype(np.uint8)
axes[1].imshow(display_img)
axes[1].set_title("Image normalisée")
axes[1].axis('on')

plt.tight_layout()
plt.show()

**Test Kernels for Convolution**

Before using the real CNN coefficients, we can test convolution with well-known kernels to verify functionality:

1. **Identity Kernel** (no change - sanity check)
- Output should be identical to input
- Verifies that convolution implementation doesn't introduce artifacts

2. **Edge Detection Kernels**
- **Horizontal edges**: Detects horizontal features
- **Vertical edges**: Detects vertical features
- **Sobel**: Detects edges in specific directions
- Easy to verify visually - edges should be highlighted

3. **Blur/Smoothing Kernels**
- **Box blur**: Simple averaging
- **Gaussian blur**: Smooth blurring
- Output should be blurred/softened

4. **Sharpening Kernel**
- Enhances edges and details
- Output should look crisper

In [ ]:
# Simple 2D convolution implementation (for single channel)
def convolve2d(image, kernel, padding='valid'):
    """
    Apply 2D convolution to a single channel image.
    
    Parameters:
    - image: 2D numpy array (single channel)
    - kernel: 2D numpy array (3x3)
    - padding: 'valid' (no padding) or 'same' (zero padding)
    
    Returns:
    - Convolved image
    """
    kernel_size = kernel.shape[0]
    pad_size = kernel_size // 2
    
    if padding == 'same':
        # Add zero padding
        padded_img = np.pad(image, pad_size, mode='constant', constant_values=0)
    else:
        padded_img = image
    
    # Output dimensions
    if padding == 'same':
        out_h, out_w = image.shape
    else:
        out_h = image.shape[0] - kernel_size + 1
        out_w = image.shape[1] - kernel_size + 1
    
    output = np.zeros((out_h, out_w), dtype=np.float32)
    
    # Perform convolution
    for i in range(out_h):
        for j in range(out_w):
            # Extract region
            region = padded_img[i:i+kernel_size, j:j+kernel_size]
            # Element-wise multiply and sum
            output[i, j] = np.sum(region * kernel)
    
    return output

# Apply convolution to RGB image (apply same kernel to each channel)
def convolve_rgb(image, kernel, padding='same'):
    """
    Apply 2D convolution to RGB image (same kernel for all channels).
    """
    output = np.zeros_like(image, dtype=np.float32)
    for c in range(3):  # For each RGB channel
        output[:, :, c] = convolve2d(image[:, :, c], kernel, padding)
    return output

In [ ]:
# Define test kernels (3x3)
test_kernels = {
    'Identity': np.array([
        [0, 0, 0],
        [0, 1, 0],
        [0, 0, 0]
    ], dtype=np.float32),
    
    'Edge Detection (Horizontal)': np.array([
        [-1, -1, -1],
        [ 0,  0,  0],
        [ 1,  1,  1]
    ], dtype=np.float32),
    
    'Edge Detection (Vertical)': np.array([
        [-1, 0, 1],
        [-1, 0, 1],
        [-1, 0, 1]
    ], dtype=np.float32),
    
    'Sobel X (Vertical Edges)': np.array([
        [-1, 0, 1],
        [-2, 0, 2],
        [-1, 0, 1]
    ], dtype=np.float32),
    
    'Sobel Y (Horizontal Edges)': np.array([
        [-1, -2, -1],
        [ 0,  0,  0],
        [ 1,  2,  1]
    ], dtype=np.float32),
    
    'Box Blur': np.array([
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]
    ], dtype=np.float32) / 9.0,
    
    'Sharpen': np.array([
        [ 0, -1,  0],
        [-1,  5, -1],
        [ 0, -1,  0]
    ], dtype=np.float32),
    
    'Emboss': np.array([
        [-2, -1, 0],
        [-1,  1, 1],
        [ 0,  1, 2]
    ], dtype=np.float32),
}

# Display all kernels
print("Test Kernels for Convolution:\n")
for name, kernel in test_kernels.items():
    print(f"{name}:")
    print(kernel)
    print(f"Sum: {kernel.sum():.2f} (important for brightness preservation)\n")

In [ ]:
# Test convolution with different kernels
# Select which kernels to visualize
kernels_to_test = ['Identity', 'Edge Detection (Horizontal)', 'Sobel X (Vertical Edges)', 'Box Blur', 'Sharpen']

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

# Original image
axes[0].imshow(img)
axes[0].set_title("Original Image")
axes[0].axis('on')

# Apply each kernel
for idx, kernel_name in enumerate(kernels_to_test, start=1):
    kernel = test_kernels[kernel_name]
    
    # Apply convolution
    result = convolve_rgb(img, kernel, padding='same')
    
    # Clip and convert for display
    # For edge detection: shift and scale to [0, 255]
    if 'Edge' in kernel_name or 'Sobel' in kernel_name:
        display_result = np.abs(result)  # Take absolute value
        display_result = (display_result / display_result.max() * 255).astype(np.uint8)
    else:
        # Clip to valid range
        display_result = np.clip(result, 0, 255).astype(np.uint8)
    
    axes[idx].imshow(display_result)
    axes[idx].set_title(f"{kernel_name}")
    axes[idx].axis('on')

plt.tight_layout()
plt.show()

print("\n✓ Convolution test complete!")
print("\nWhat to verify:")
print("- Identity: Should look identical to original")
print("- Edge Detection: Should highlight edges (horizontal/vertical)")
print("- Blur: Should look softer/blurred")
print("- Sharpen: Should enhance details and edges")